In [1]:
import pandas as pd
import numpy as np
import spacy
import os
import re
import time
import errno
import os
import signal
import functools
from textblob import TextBlob
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,SnowballStemmer
import openai

/Users/henry/miniforge3/envs/mlenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Combining the scrapped data 
# THIS NOTEBOOK CANNOT BE RUNNED WITHOUT AN OPENAI API KEY

In [2]:
id_review = []
caption = []
relative_date = []
retrieval_date = []
rating = []
username = []
n_review_user = []
n_photo_user = []
url_user = []
restraunt_name = []

for csv in os.listdir("rawdata"):
    path =  "./rawdata/" + csv
    name =  csv.replace(".csv","")

    d = pd.read_csv(path)
    id_review.extend(d.id_review)
    caption.extend(d.caption)
    relative_date.extend(d.relative_date)
    retrieval_date.extend(d.retrieval_date)
    rating.extend(d.rating)
    username.extend(d.username)
    n_review_user.extend(d.n_review_user)
    n_photo_user.extend(d.n_photo_user)
    url_user.extend(d.url_user)
    restraunt_name.extend([name] * len(d.id_review))



df = pd.DataFrame({
    "id_review" : id_review,
    "caption": caption,
    "relative_date" : relative_date,
    "retrieval_date" : retrieval_date,
    "rating" : rating,
    "username" : username,
    "n_review_user" : n_review_user,
    "n_photo_user": n_photo_user,
    "url_user": url_user,
    "restraunt_name" : restraunt_name
})

df

id_review  \
0      ChZDSUhNMG9nS0VJQ0FnSURoeUxINVhnEAE   
1     ChdDSUhNMG9nS0VJQ0FnSURoX1phbWlRRRAB   
2     ChdDSUhNMG9nS0VJQ0FnSURoN2IzU2hRRRAB   
3      ChZDSUhNMG9nS0VJQ0FnSURobWNQbVd3EAE   
4      ChZDSUhNMG9nS0VJQ0FnSURoc2JEV0hnEAE   
...                                    ...   
6063  ChdDSUhNMG9nS0VJQ0FnSUN3dDYtamxBRRAB   
6064   ChZDSUhNMG9nS0VJQ0FnSURBdDlpWFhREAE   
6065  ChdDSUhNMG9nS0VJQ0FnSUNndnJYWTZRRRAB   
6066  ChdDSUhNMG9nS0VJQ0FnSUNBbnNPaDN3RRAB   
6067   ChZDSUhNMG9nS0VJQ0FnSUNnLUluSEl3EAE   

                                                caption relative_date  \
0                                                   NaN     a day ago   
1                                                   NaN    4 days ago   
2     Took my son there for dinner and we had a good...    5 days ago   
3                                                   NaN    a week ago   
4                                                   NaN    a week ago   
...                                                 ...           ...   
6063                                                NaN   5 years ago   
6064                                                NaN   5 years ago   
6065                                                NaN   5 years ago   
6066  Some hits and some misses. The french toast wa...   5 years ago   
6067  Nice interiors with a cozy atmosphere. I tried...   5 years ago   

                  retrieval_date  rating          username  n_review_user  \
0     2023-03-20 21:28:13.948446     4.0   James Bickerton              0   
1     2023-03-20 21:28:13.949708     4.0         Marcus Oh              0   
2     2023-03-20 21:28:13.951090     5.0           laurent              0   
3     2023-03-20 21:28:13.952226     3.0             kushi              0   
4     2023-03-20 21:28:13.953282     5.0  Kalinda Nawalage              0   
...                          ...     ...               ...            ...   
6063  2023-03-21 11:35:01.638271     3.0        Audrey Lee              0   
6064  2023-03-21 11:35:01.639011     5.0      Tsikenu Phua              0   
6065  2023-03-21 11:35:01.639743     5.0        Mimi Yahya              0   
6066  2023-03-21 11:35:01.640495     4.0     Shalini Singh              0   
6067  2023-03-21 11:35:01.641293     4.0      Shuvam Nandi              0   

      n_photo_user                                           url_user  \
0                0  https://www.google.com/maps/contrib/1075687804...   
1                0  https://www.google.com/maps/contrib/1102131170...   
2                0  https://www.google.com/maps/contrib/1052889272...   
3                0  https://www.google.com/maps/contrib/1009673441...   
4                0  https://www.google.com/maps/contrib/1154990112...   
...            ...                                                ...   
6063             0  https://www.google.com/maps/contrib/1081779193...   
6064             0  https://www.google.com/maps/contrib/1096226656...   
6065             0  https://www.google.com/maps/contrib/1056346676...   
6066             0  https://www.google.com/maps/contrib/1143218590...   
6067             0  https://www.google.com/maps/contrib/1041464017...   

       restraunt_name  
0             Hooters  
1             Hooters  
2             Hooters  
3             Hooters  
4             Hooters  
...               ...  
6063  PennyUniversity  
6064  PennyUniversity  
6065  PennyUniversity  
6066  PennyUniversity  
6067  PennyUniversity  

[6068 rows x 10 columns]

In [3]:
# removing empty comments
df = df[df.caption.isna() == False].reset_index(drop = True)
df

id_review  \
0     ChdDSUhNMG9nS0VJQ0FnSURoN2IzU2hRRRAB   
1      ChZDSUhNMG9nS0VJQ0FnSURodHVDeWRREAE   
2      ChZDSUhNMG9nS0VJQ0FnSURoLW9HUEhREAE   
3      ChZDSUhNMG9nS0VJQ0FnSURoaXVyTkhREAE   
4      ChZDSUhNMG9nS0VJQ0FnSURock9MV1pREAE   
...                                    ...   
3887   ChZDSUhNMG9nS0VJQ0FnSURRbXFLWVlBEAE   
3888  ChdDSUhNMG9nS0VJQ0FnSUNRM2EyYTJnRRAB   
3889   ChZDSUhNMG9nS0VJQ0FnSUNBX0kzYlpnEAE   
3890  ChdDSUhNMG9nS0VJQ0FnSUNBbnNPaDN3RRAB   
3891   ChZDSUhNMG9nS0VJQ0FnSUNnLUluSEl3EAE   

                                                caption relative_date  \
0     Took my son there for dinner and we had a good...    5 days ago   
1     Pathetic food. Ordered vegetarian burger. Tast...    a week ago   
2     Friendly staff, superb service, food was serve...    a week ago   
3     I have yet to find a Hooter's that I didn't li...   2 weeks ago   
4     Alcohol is expensive everywhere in Singapore, ...   2 weeks ago   
...                                                 ...           ...   
3887                     My favorite is muddy espresso.   5 years ago   
3888  Our order taker was a bit blur and accidentall...   5 years ago   
3889  Went for lunch, a bit so-so, didnt try the spe...   5 years ago   
3890  Some hits and some misses. The french toast wa...   5 years ago   
3891  Nice interiors with a cozy atmosphere. I tried...   5 years ago   

                  retrieval_date  rating         username  n_review_user  \
0     2023-03-20 21:28:13.951090     5.0          laurent              0   
1     2023-03-20 21:28:13.954334     1.0           Fa Ris              0   
2     2023-03-20 21:28:13.955274     5.0   Florencio Gali              0   
3     2023-03-20 21:28:13.956210     5.0   Joshua Manning              0   
4     2023-03-20 21:28:13.957351     4.0          J.K LEE              0   
...                          ...     ...              ...            ...   
3887  2023-03-21 11:35:01.635093     4.0        Luna Choi              0   
3888  2023-03-21 11:35:01.636697     2.0      Shawn Galey              0   
3889  2023-03-21 11:35:01.637496     4.0  Antonius Howard              0   
3890  2023-03-21 11:35:01.640495     4.0    Shalini Singh              0   
3891  2023-03-21 11:35:01.641293     4.0     Shuvam Nandi              0   

      n_photo_user                                           url_user  \
0                0  https://www.google.com/maps/contrib/1052889272...   
1                0  https://www.google.com/maps/contrib/1061823533...   
2                0  https://www.google.com/maps/contrib/1074077044...   
3                0  https://www.google.com/maps/contrib/1083143598...   
4                0  https://www.google.com/maps/contrib/1101703195...   
...            ...                                                ...   
3887             0  https://www.google.com/maps/contrib/1156646053...   
3888             0  https://www.google.com/maps/contrib/1032833126...   
3889             0  https://www.google.com/maps/contrib/1011209020...   
3890             0  https://www.google.com/maps/contrib/1143218590...   
3891             0  https://www.google.com/maps/contrib/1041464017...   

       restraunt_name  
0             Hooters  
1             Hooters  
2             Hooters  
3             Hooters  
4             Hooters  
...               ...  
3887  PennyUniversity  
3888  PennyUniversity  
3889  PennyUniversity  
3890  PennyUniversity  
3891  PennyUniversity  

[3892 rows x 10 columns]

In [21]:
#df.to_csv("/Users/henry/Desktop/year 4 sem 1/BT4222/Project2/data/raw_unlabelled.csv")

# Segregating Aspects and relevant phrases/words describing that aspect (Aspect Extraction)
### Aspects we are considering 

* Food
* Service
* Ambience
* Others

# Aspect-Extraction
### THIS NOTEBOOK CANNOT BE RUNNED WITHOUT AN API KEY

In [17]:

openai.api_key = "" # api key


class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wrapper

    return decorator

@timeout(10)
def extract_aspect(text):
  d = {}
  response = openai.Completion.create(
      model="text-davinci-003",
      prompt=f"extract all parts of text associated with food quality or service quality of ambience quality (can be NA). reply in the form  [food: all parts of text , svc: all parts of text , amb: all parts of text] \n\n' '{text}'",
      temperature=0.5,
      max_tokens=512,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
  text = response["choices"][0]["text"]
  

  # results will be smth like [food: all answers, service: all answers, ambience: all answers] 
  # seperate the parts

  food_part = re.search("food: ([^;]*) svc",text).group(0)[6:-5]
  service_part = re.search("svc: ([^;]*) amb",text).group(0)[5:-5]
  ambience_part = re.search("amb: ([^;]*)",text).group(0)[5: -1]
 
  return food_part, service_part, ambience_part


#example

print("Example review:")
example = df.caption[6]
# print the text out nicely,seperate by sentences
x = " "
for i in example:
    if i != ".":
        x += i
    else:
        print(x)
        x = ""
print("------------------")
extract_aspect(df.caption[6])


Example review:
 After their renovation, the food quality has dropped significantly
 The calamari was cold and greasy, without salt
 The music was boring, being played on a poor quality music system in a corner
------------------


('After their renovation, the food quality has dropped significantly. The calamari was cold and greasy, without salt. ',
 'NA ',
 'The music was boring, being played on a poor quality music system in a corner.')

In [263]:
food_part = []
service_part = []
ambience_part = []

count = 0
retry_counts = 0

while count < len(df.caption):
    text = df.caption[count]
    print(text)
    if retry_counts == 5:
        count += 1
        food_part.append(np.nan)
        service_part.append(np.nan)
        ambience_part.append(np.nan)
    try:
        food,svc,ambience = extract_aspect(text)
        food_part.append(food)
        service_part.append(svc)
        ambience_part.append(ambience)
        count += 1
        retry_counts = 0
    except AttributeError:
        time.sleep(2)
        retry_counts += 1
        continue
    except TimeoutError:
        time.sleep(2)
        continue

Took my son there for dinner and we had a good time. We both ate a Philly cheese steak and it was really good. Service was good, girls are very pleasant and pay attention to the service, prices are very reasonable.
Pathetic food. Ordered vegetarian burger. Tasted awful. Potato wedges were cut so big and didn’t taste good.  Also beware that happy hour beer costs more than regular ones.
Friendly staff, superb service, food was serve hot and fresh, not so pricey, pretty service staff. Place is a bit outdated might needs some upgrading. But overall experience is 5star
I have yet to find a Hooter's that I didn't like. Cold beer, hot wings.
Alcohol is expensive everywhere in Singapore, so I went to Hooters. The chicken is delicious and there are things to see. There was a beer 2+1 promotion, and it was nice to take pictures with the staff as if I was going to a Japanese Mate Cafe.
Lovely bar, super friendly staff and smiling too. Good, as usual, was lovely, good offers on beer BOGO or BTGO. 

In [264]:
aspect = {"food_aspect": food_part,
          "service_aspect" : service_part,
          "ambience_aspect" : ambience_part}

aspect = pd.DataFrame(aspect)
aspect

food_aspect  \
0                   Philly cheese steak was really good   
1     Pathetic food, Ordered vegetarian burger, tast...   
2          food was serve hot and fresh, not so pricey,   
3                                  Cold beer, hot wings   
4                              The chicken is delicious   
...                                                 ...   
3887                           The food was really good   
3888  'When the food came they refused to credit us ...   
3889       a bit so-so, didn't try the specialty coffee   
3890  French toast was nice but too sweet, coffee wa...   
3891  Turkish poached eggs with yoghurt and marissa,...   

                                         service_aspect  \
0     Service was good, girls are very pleasant and ...   
1                                                    NA   
2     Friendly staff, superb service, pretty service...   
3                                                    NA   
4     There was a beer 2+1 promotion, it was nice to...   
...                                                 ...   
3887                 the service was excellent, amb: th   
3888  'Our order taker was a bit blur and accidental...   
3889                                                 NA   
3890                                                 NA   
3891                                                 NA   

                                        ambience_aspect  
0                                                    NA  
1                                                    NA  
2     Place is a bit outdated might needs some upgra...  
3                                                    NA  
4                                                    NA  
...                                                 ...  
3887                the ambiance was cozy and inviting.  
3888  'I come here a lot and used to think that the ...  
3889                                                 NA  
3890                                                 NA  
3891              Nice interiors with a cozy atmosphere  

[3892 rows x 3 columns]

In [281]:
# some parts of the response is has problems 
# the model intepreted the question too literally and returned "all parts of text"
# we replace those values and those with "NA" as value with NaN's
aspect.loc[(aspect.food_aspect == "all parts of text") | (aspect.food_aspect == "NA"), "food_aspect"] = np.nan
aspect.loc[(aspect.service_aspect == "all parts of text") | (aspect.service_aspect == "NA"), "service_aspect"] = np.nan
aspect.loc[(aspect.ambience_aspect == "all parts of text") | (aspect.ambience_aspect == "NA"), "ambience_aspect"] = np.nan

In [ ]:
#pd.concat([df,aspect], axis= 1).to_csv("/Users/henry/Desktop/year 4 sem 1/BT4222/Project2/data/extracted_aspects.csv")

In [286]:
#aspect.to_csv("/Users/henry/Desktop/year 4 sem 1/BT4222/Project2/data/extracted_aspects_raw.csv")

In [ ]:
#aspect[aspect.ambience_aspect.isna()]

food_aspect  \
0                   Philly cheese steak was really good   
1     Pathetic food, Ordered vegetarian burger, tast...   
3                                  Cold beer, hot wings   
4                              The chicken is delicious   
9       Very good food, Slush of the day, I had orange.   
...                                                 ...   
3882  Some of the best coffee in Singapore. Menu is ...   
3883                                      mediocre food   
3885                                     Muddy Espresso   
3889       a bit so-so, didn't try the specialty coffee   
3890  French toast was nice but too sweet, coffee wa...   

                                         service_aspect ambience_aspect  
0     Service was good, girls are very pleasant and ...             NaN  
1                                                   NaN             NaN  
3                                                   NaN             NaN  
4     There was a beer 2+1 promotion, it was nice to...             NaN  
9      best place to wait, till the rivercruise starts.             NaN  
...                                                 ...             ...  
3882                                                NaN             NaN  
3883                             extremely slow service             NaN  
3885                                                NaN             NaN  
3889                                                NaN             NaN  
3890                                                NaN             NaN  

[2013 rows x 3 columns]